# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [14]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, f1_score

In [15]:
data = pd.read_csv('data/labeled.csv')

In [33]:
class Experiment:

    train, test = train_test_split(data, test_size=0.1, shuffle=True)
    train.reset_index(inplace=True)
    test.reset_index(inplace=True)

    def __init__(
            self,
            vectorizer_model: object,
            vectorizer_args : dict,
            classifier_model: object,
            classifier_args : dict
        ):
        
        self.vectorizer = vectorizer_model(**vectorizer_args)
        X = self.vectorizer.fit_transform(self.train.comment)
        y = self.train.toxic.values

        self.classifier = classifier_model(**classifier_args)
        self.classifier.fit(X, y)

    def get_test_preds(self):
        X = self.vectorizer.transform(self.test.comment)
        preds = self.classifier.predict(X)
        return preds

    def get_report(self):
        return classification_report(
            self.test.toxic.values,
            self.get_test_preds(),
            zero_division=0
        )
    
    def get_f1(self):
        return f1_score(
            self.test.toxic.values,
            self.get_test_preds(),
            zero_division=0        
        )

In [34]:
experiment_default = Experiment(
    TfidfVectorizer,
    {},
    MultinomialNB,
    {}
)

In [35]:
print(
    experiment_default.get_report()
)

              precision    recall  f1-score   support

         0.0       0.72      1.00      0.83       936
         1.0       0.97      0.27      0.43       506

    accuracy                           0.74      1442
   macro avg       0.84      0.64      0.63      1442
weighted avg       0.81      0.74      0.69      1442



In [36]:
import razdel

In [37]:
## Если просто сунуть по ссылке, прилетает TypeError: '<' not supported between instances of 'Substring' and 'Substring'

def odel_tokenize(text):
    return [_.text for _ in razdel.tokenize(text)]

In [38]:
experiment_razdel = Experiment(
    TfidfVectorizer,
    {
        "tokenizer": odel_tokenize,
        "analyzer" : "word"
    },
    MultinomialNB,
    {}
)

d:\HSE subj data\актуальные проблемы компьютерной лингвистики\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [39]:
print(
    experiment_razdel.get_report()
)

              precision    recall  f1-score   support

         0.0       0.70      1.00      0.82       936
         1.0       0.96      0.19      0.32       506

    accuracy                           0.71      1442
   macro avg       0.83      0.59      0.57      1442
weighted avg       0.79      0.71      0.64      1442



#### Комментарий

В документации Razdel написано, что

<q>
Правила в Razdel оптимизированы для аккуратно написанных текстов с правильной пунктуацией. Решение хорошо работает с новостными статьями, художественными текстами. На постах из социальных сетей, расшифровках телефонных разговоров качество ниже.
</q>

Поэтому неудивительно, что на данном датасете Razdel работает чуть хуже.

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [40]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [41]:
experiment_params = (

    {
        "classifier_model": MultinomialNB,
        "classifier_args" : {},  ##  Чудесным образом любое вмешательство в парамеры сильно ухудшает результаты
        "vectorizer_model": CountVectorizer,
        ##  Здесь и далее наилучшие результаты давала векторизация с параметрами analyzer + ngram_range + max_df
        ##  Т.е. с 3-мя вручную заданными параметрами, поэтому 2 отставшихся по заданию забил дефолтными.
        ##  Иные вмешательства делали результат хуже.
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

    ##  Фактически задание не выполнено т.к. в обоих случаях результаты получаются лучше с мешком слов.
    ##  Однако вот такая конфигурация проходит только с варнингом, поэтому можно считать, что TF-IDF лучше т.к. стабильнее (?)
    {   
        "classifier_model": LogisticRegression,
        "classifier_args" : {"solver":"liblinear", "penalty":"l2"},
        "vectorizer_model": CountVectorizer,
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

    {
        "classifier_model": LogisticRegression,
        "classifier_args" : {"solver":"liblinear", "penalty":"l2"},
        "vectorizer_model": TfidfVectorizer,
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

)

In [42]:
finished_experiments = []

for d in experiment_params:
    experiment = Experiment(**d)
    print(f"{d["classifier_model"].__name__}\t{d["vectorizer_model"].__name__}")
    print("F1: ", experiment.get_f1())
    print(experiment.get_report())
    finished_experiments.append(experiment)

MultinomialNB	CountVectorizer
F1:  0.8678861788617886
              precision    recall  f1-score   support

         0.0       0.92      0.95      0.93       936
         1.0       0.89      0.84      0.87       506

    accuracy                           0.91      1442
   macro avg       0.91      0.89      0.90      1442
weighted avg       0.91      0.91      0.91      1442



d:\HSE subj data\актуальные проблемы компьютерной лингвистики\.venv\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LogisticRegression	CountVectorizer
F1:  0.8530318602261048
              precision    recall  f1-score   support

         0.0       0.91      0.94      0.93       936
         1.0       0.89      0.82      0.85       506

    accuracy                           0.90      1442
   macro avg       0.90      0.88      0.89      1442
weighted avg       0.90      0.90      0.90      1442

LogisticRegression	TfidfVectorizer
F1:  0.819672131147541
              precision    recall  f1-score   support

         0.0       0.87      0.96      0.92       936
         1.0       0.92      0.74      0.82       506

    accuracy                           0.89      1442
   macro avg       0.90      0.85      0.87      1442
weighted avg       0.89      0.89      0.88      1442



In [43]:
##  Для читаемости
nb_bow_preds       = finished_experiments[0].get_test_preds()
logreg_bow_preds   = finished_experiments[1].get_test_preds()
logreg_tfidf_preds = finished_experiments[2].get_test_preds()

In [44]:
same_normal_results      = []
same_toxic_results       = []
different_normal_results = []
different_toxic_results  = []

# for comment, is_toxic, nb_bow_pred, logreg_bow_pred, logreg_tfidf_pred \
for comment, results in zip(
        Experiment.test.comment,
        zip(
            experiment.test.toxic,
            nb_bow_preds,
            logreg_bow_preds,
            logreg_tfidf_preds
        )
    ):

    elem_to_append = comment.strip() + ": " + " ".join(map(str, results))
    if sum(results) in (0, 4):
        if results[0] == 1:
            same_toxic_results.append(elem_to_append)
        else:
            same_normal_results.append(elem_to_append)
    else:
        if results[0] == 1:
            different_toxic_results.append(elem_to_append)
        else:
            different_normal_results.append(elem_to_append)

In [45]:
len(same_toxic_results), len(same_normal_results), len(different_toxic_results), len(different_normal_results)

(358, 863, 148, 73)

In [46]:
for c in same_toxic_results[:10]:
    print(c)

Может верблюдов сжечь? Во славу и величие Израиля, к примеру.: 1.0 1.0 1.0 1.0
лол, вафлёрные зоошизики меня минусуют) пздц убогие: 1.0 1.0 1.0 1.0
Моему бате отвалилось и упало на голову. Кровища была, потом шишка осталась (да, я сволочь ): 1.0 1.0 1.0 1.0
Он платить за хохлов не будет.: 1.0 1.0 1.0 1.0
ты заткнуться можешь?: 1.0 1.0 1.0 1.0
Я вам хамил? Я это не заметил. Или это из серии ажемать яжотец яжбэмэвэ?: 1.0 1.0 1.0 1.0
Прям, как в песне группы DZIDZIO Павук З ла лахудра Бодьку-павука. А мораль такова з ла баба мужика!: 1.0 1.0 1.0 1.0
Высирается твоя мамаша тебе на лицо по утрам, а я высказался, обессилев от негодования от засравших политач ловцов лахты под кроватью.: 1.0 1.0 1.0 1.0
Слишком, блядь, мощно: 1.0 1.0 1.0 1.0
мааам, меня обидили, почему я дегенерат, мам?: 1.0 1.0 1.0 1.0


In [47]:
for c in same_normal_results[:10]:
    print(c)

Причем тут латынь? Есть название на русском языке, современное и устоявщееся и подкрепленное современной докумениацией этой страны.. По какой-то причине в других языках называют города не транслитизированным переносом, а как-то иначе. Если это общепринятые нормы, то почему надо спорить ламборгини или ламборджини, если это общепринятая норма?: 0.0 0.0 0.0 0.0
Вот зря вы. У нас по сравнению с США огромный опыт работы с орбитальными станциями (доставка грузов космонавтов, постройка и обслуживание станций и т.д.) и огромный опыт по космическим кораблям. Да, многие технологии сейчас уже давно впереди (вычислительные, ПО и т.п.). Но если совместить наш опыт с технологиями зарубежных стран, то получиться реально стоящие косм. корабли и косм. станции, благодаря которым человечество будет спокойно осваивать другие планеты и их спутники. Но опять же, мы ни при каких условиях с кем-то работать не будем, ибо этот огромный опыт, который мы передадим -- может пойти не в то русло. Так что если если и

In [48]:
for c in different_toxic_results[:10]:
    print(c)

Забыл добавить , если при этом чурка пожизненно в сибири ,лес валить!: 1.0 1.0 0.0 0.0
Беда в том, что полиции почему-то похуй на это быдлана Он сам по себе довольно таки темный в плане дел с законом. Не знаю, правда правое дело делает или там огромная крыша, но полиция так и не будет его трогать.: 1.0 0.0 1.0 0.0
В те времена и мотобайкеров - рокерами называли...: 1.0 0.0 0.0 0.0
Шинка, ты чего без аватарки сегодня?: 1.0 0.0 1.0 1.0
Да всё гораздо проще. Плешивый карлан интересами граждан никогда не интересовался. Отсюда ебовый обвал рубля, чтобы холопы батрахили за зп в 2 раза меньше, чем раньше. Отмена пенсий (батрачить 40 лет, чтобы 4 года получать пенсию). Вот уже начал на интернет замахиваться, чтобы ради безопасности граждан они получали только одобренную режимом информацию. Настоящий анон, всегда был на своей стороне. Ему всегда было похуй на ракеты. Анон свободолюбив даже сидя у мамки на шее. Так что за плешивого карлика могут топить только враги анона, которые сидят тут на за

In [49]:
for c in different_normal_results[:10]:
    print(c)

ряженые кизяки Извинись!: 0.0 1.0 1.0 1.0
Носить телесные носки. Неграм проще.: 0.0 0.0 1.0 0.0
Военные Базы!!!!!!! Ты что не понимаишь!!!!!! Военные!!!!!!!!! Кругом война! Никаких прочих идей не существует. А в школе - вам нагло врут!: 0.0 1.0 1.0 1.0
I believe I can touch the sky: 0.0 1.0 0.0 0.0
хохохо, а ты хорош! В: 0.0 1.0 1.0 1.0
Как что-то плохое. после фоток Бэллы ИТТ хоть будет на что потилибонькать: 0.0 1.0 0.0 0.0
Аааа, ясно как ты знаешь США, наверное по твоему тут и негров линчуют А я своими глазами видел в прошлом году гей парад, который не то что разгоняли, там участвовали геи-копы, геи-политики, геи-пожарные, геи-медики итп Поэтому не вижу смысла продолжать спорить с человеком, который знает об Америке только с заголовков газет, лучше полюбуйся на фотки гей парадов в США на которых я был: 0.0 1.0 1.0 1.0
Нихуя, забытый. Он недавно лям собрал: 0.0 1.0 0.0 1.0
http: www.aspd.ru napravlenija dejatelnosti akusticheskie-materialy akusticheskaja rezina gsp 100 vibrozaschita:

#### Комментарий

Токсичность комментариев -- очень субъективная тема, особенно в рамках бинарной классификации. Кого-то может и вот такое задеть: ```"хочешь грудь модератора?"``` <s>(последний пример из списка выше)</s> <i>перезапустил все ячейки и результаты поменялись, но раньше этот пример там был. Надо приучить себя определять рандом сид.</i> Конечно, для такой задачи нужно бы иметь побольше классов.

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [50]:
experiment_params = (

    {
        "classifier_model": MultinomialNB,
        "classifier_args" : {},  ##  Чудесным образом любое вмешательство в парамеры сильно ухудшает результаты
        "vectorizer_model": CountVectorizer,
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

    {   
        "classifier_model": LogisticRegression,
        "classifier_args" : {"solver":"liblinear", "penalty":"l2"},
        "vectorizer_model": CountVectorizer,
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

    {
        "classifier_model": DecisionTreeClassifier,
        "classifier_args" : {},  ##  Чудесным образом любое вмешательство в парамеры сильно ухудшает результаты
        "vectorizer_model": CountVectorizer,
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

    {
        "classifier_model": RandomForestClassifier,
        "classifier_args" : {"n_estimators": 10},  ##  Чудесным образом любое вмешательство в парамеры сильно ухудшает результаты
        "vectorizer_model": CountVectorizer,
        "vectorizer_args" : {"analyzer": "char_wb","ngram_range": (3, 5), "max_df": 0.48, "encoding":"utf-8", "input":"content"}
    },

)

In [51]:
finished_experiments = []

for d in experiment_params:
    experiment = Experiment(**d)
    print(f"{d["classifier_model"].__name__}\t{d["vectorizer_model"].__name__}")
    print("F1: ", experiment.get_f1())
    print(experiment.get_report())
    finished_experiments.append(experiment)

MultinomialNB	CountVectorizer
F1:  0.8678861788617886
              precision    recall  f1-score   support

         0.0       0.92      0.95      0.93       936
         1.0       0.89      0.84      0.87       506

    accuracy                           0.91      1442
   macro avg       0.91      0.89      0.90      1442
weighted avg       0.91      0.91      0.91      1442



d:\HSE subj data\актуальные проблемы компьютерной лингвистики\.venv\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LogisticRegression	CountVectorizer
F1:  0.8530318602261048
              precision    recall  f1-score   support

         0.0       0.91      0.94      0.93       936
         1.0       0.89      0.82      0.85       506

    accuracy                           0.90      1442
   macro avg       0.90      0.88      0.89      1442
weighted avg       0.90      0.90      0.90      1442

DecisionTreeClassifier	CountVectorizer
F1:  0.6915113871635611
              precision    recall  f1-score   support

         0.0       0.82      0.87      0.84       936
         1.0       0.73      0.66      0.69       506

    accuracy                           0.79      1442
   macro avg       0.78      0.76      0.77      1442
weighted avg       0.79      0.79      0.79      1442

RandomForestClassifier	CountVectorizer
F1:  0.635
              precision    recall  f1-score   support

         0.0       0.78      0.96      0.86       936
         1.0       0.86      0.50      0.64       506

    accura